In [ ]:
#!/usr/bin/env python
import os
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

TRADING_DAYS = 252

# ============================================================
# CONFIG
# ============================================================
WFO_EQUITY_FILE = "./walkforward_top10_by_cagr_equity_curve.csv"  # WFO output
SPY_FILE = "./8-SPY_200DMA_market_regime/8-SPY_200DMA_regime.parquet"  # optional benchmark

OUT_DIR = "./13d-monte_carlo_output_wfo"
os.makedirs(OUT_DIR, exist_ok=True)

START_CAPITAL = 342000  # <-- scale results to dollars
N_SIMS = 10000
BLOCK_SIZE = 25
SEED = 123

DD_THRESHOLD = 0.50
LATE_GROWTH_MULT = 1.6
LATE_DD_FRAC = 0.40

# ============================================================
# METRICS
# ============================================================
def cagr_from_path(values, start_date, end_date):
    values = np.asarray(values, dtype=float)
    if len(values) < 2 or values[0] <= 0:
        return np.nan
    years = (pd.Timestamp(end_date) - pd.Timestamp(start_date)).days / 365.25
    if years <= 0:
        return np.nan
    return (values[-1] / values[0]) ** (1.0 / years) - 1.0

def ann_sharpe(returns):
    r = np.asarray(returns, dtype=float)
    std = np.std(r, ddof=1)
    if std <= 0 or np.isnan(std):
        return np.nan
    return np.sqrt(TRADING_DAYS) * np.mean(r) / std

def max_drawdown(values):
    arr = np.asarray(values, dtype=float)
    peaks = np.maximum.accumulate(arr)
    dd = arr / peaks - 1.0
    return float(np.min(dd))

def block_bootstrap(returns, n, block_size, rng):
    r = np.asarray(returns, dtype=float)
    L = len(r)
    if L <= 0 or n <= 0:
        return np.array([], dtype=float)
    if block_size <= 0:
        raise ValueError("BLOCK_SIZE must be > 0")
    if block_size > L:
        raise ValueError(f"BLOCK_SIZE ({block_size}) cannot exceed number of returns ({L})")

    out = []
    while len(out) < n:
        i = int(rng.integers(0, L - block_size + 1))
        out.extend(r[i:i + block_size])
    return np.array(out[:n], dtype=float)

def two_sided_p_from_null(null_dist, observed):
    # P(|X| >= |obs|)
    null_dist = np.asarray(null_dist, float)
    return float(np.mean(np.abs(null_dist) >= abs(observed)))

# ============================================================
# LOAD WFO EQUITY
# ============================================================
def load_wfo_equity(path: str | Path) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"Missing WFO equity file: {path}")

    if path.suffix.lower() == ".parquet":
        eq = pd.read_parquet(path)
    else:
        eq = pd.read_csv(path)

    if "date" not in eq.columns:
        raise ValueError("WFO equity file must have a 'date' column")

    if "equity" in eq.columns:
        val_col = "equity"
    elif "portfolio_value" in eq.columns:
        val_col = "portfolio_value"
    else:
        raise ValueError("WFO equity file must have 'equity' or 'portfolio_value'")

    eq = eq.copy()
    eq["date"] = pd.to_datetime(eq["date"])
    eq[val_col] = pd.to_numeric(eq[val_col], errors="coerce")
    eq = eq.dropna(subset=["date", val_col]).sort_values("date")
    eq = eq.drop_duplicates(subset=["date"]).reset_index(drop=True)
    eq = eq[eq[val_col] > 0].reset_index(drop=True)
    eq = eq.rename(columns={val_col: "equity"})
    return eq[["date", "equity"]]

print("=== Loading WFO equity curve ===")
equity = load_wfo_equity(WFO_EQUITY_FILE)
if len(equity) < 3:
    raise ValueError("Not enough equity observations after cleaning (need at least 3 rows).")

# Scale normalized equity to dollars
equity0 = float(equity["equity"].iloc[0])
scale = START_CAPITAL / equity0
equity["equity_usd"] = equity["equity"] * scale

equity = equity.sort_values("date").reset_index(drop=True)
equity["strat_ret"] = equity["equity"].pct_change().fillna(0.0)

values_usd = equity["equity_usd"].to_numpy()
values_idx = equity["equity"].to_numpy()
dates = equity["date"].to_numpy()
rets = equity["strat_ret"].to_numpy()[1:]  # drop first

n_rets = len(rets)
start_date = equity["date"].iloc[0]
end_date = equity["date"].iloc[-1]

print(f"Loaded {len(equity):,} equity points, {n_rets} return observations.")
print(f"Date range: {start_date.date()} → {end_date.date()}")
print(f"Scaled start equity: ${values_usd[0]:,.0f}")

# ============================================================
# OPTIONAL SPY ALIGNMENT (for IR)
# ============================================================
have_spy = False
ex_rets = None

try:
    spy = pd.read_parquet(SPY_FILE).copy()
    if "date" not in spy.columns:
        spy = spy.reset_index().rename(columns={"index": "date", "Date": "date"})
    spy["date"] = pd.to_datetime(spy["date"])
    if "spy_close" not in spy.columns:
        raise ValueError("SPY file missing 'spy_close'")
    spy = spy.sort_values("date")
    spy["spy_ret"] = spy["spy_close"].pct_change().fillna(0.0)

    aligned = equity.merge(spy[["date", "spy_ret"]], on="date", how="inner").dropna()
    if len(aligned) >= 3:
        have_spy = True
        ex_rets = (aligned["strat_ret"] - aligned["spy_ret"]).to_numpy()[1:]
    else:
        have_spy = False
except Exception as e:
    print(f"NOTE: SPY not loaded/aligned. IR stats disabled. Reason: {e}")
    have_spy = False

# ============================================================
# TRUE METRICS (realized WFO path)
# ============================================================
true_cagr = cagr_from_path(values_usd, start_date, end_date)
true_sharpe = ann_sharpe(rets)
true_dd = max_drawdown(values_idx)  # same as values_usd, % is invariant

print("\n=== TRUE WFO RESULTS (single realized path) ===")
print(f"CAGR:   {true_cagr:.4f}")
print(f"Sharpe: {true_sharpe:.4f}")
print(f"MaxDD:  {true_dd:.4f}")
if have_spy:
    true_ir = ann_sharpe(ex_rets)
    print(f"IR (Sharpe of excess vs SPY): {true_ir:.4f}")

# ============================================================
# MONTE CARLO (block bootstrap)
# ============================================================
print("\n=== Running Monte Carlo simulations (block bootstrap) ===")
rng = np.random.default_rng(SEED)

sim_cagrs = np.zeros(N_SIMS, float)
sim_sharpes = np.zeros(N_SIMS, float)
sim_dds = np.zeros(N_SIMS, float)

sim_max_dd_dollars = np.zeros(N_SIMS, float)
sim_peak_equity = np.zeros(N_SIMS, float)
sim_trough_equity = np.zeros(N_SIMS, float)

sim_final_equity = np.zeros(N_SIMS, float)
sim_min_equity = np.zeros(N_SIMS, float)

sim_dd50 = np.zeros(N_SIMS, float)

if have_spy:
    sim_irs = np.zeros(N_SIMS, float)

for i in range(N_SIMS):
    sim_rets = block_bootstrap(rets, n_rets, BLOCK_SIZE, rng)

    sim_curve_idx = np.cumprod(np.r_[1.0, 1.0 + sim_rets])         # normalized
    sim_curve_usd = sim_curve_idx * START_CAPITAL

    sim_final_equity[i] = sim_curve_usd[-1]
    sim_min_equity[i] = np.min(sim_curve_usd)

    running_peak = np.maximum.accumulate(sim_curve_usd)
    dd_abs = running_peak - sim_curve_usd
    dd_idx = int(np.argmax(dd_abs))
    peak_idx = int(np.argmax(sim_curve_usd[:dd_idx + 1])) if dd_idx >= 0 else 0

    sim_peak_equity[i] = sim_curve_usd[peak_idx]
    sim_trough_equity[i] = sim_curve_usd[dd_idx]
    sim_max_dd_dollars[i] = dd_abs[dd_idx]

    sim_cagrs[i] = cagr_from_path(sim_curve_usd, start_date, end_date)
    sim_sharpes[i] = ann_sharpe(sim_rets)
    sim_dds[i] = max_drawdown(sim_curve_idx)
    sim_dd50[i] = float(sim_dds[i] <= -DD_THRESHOLD)

    if have_spy:
        sim_ex = block_bootstrap(ex_rets, len(ex_rets), BLOCK_SIZE, rng)
        sim_irs[i] = ann_sharpe(sim_ex)

# ============================================================
# CONFIDENCE INTERVALS
# ============================================================
sharpe_ci_95 = np.percentile(sim_sharpes[~np.isnan(sim_sharpes)], [2.5, 50, 97.5])
print("\n=== SHARPE UNCERTAINTY (bootstrap CI) ===")
print(f"Sharpe 95% CI (2.5/50/97.5): {sharpe_ci_95[0]:.3f}, {sharpe_ci_95[1]:.3f}, {sharpe_ci_95[2]:.3f}")

if have_spy:
    ir_ci_95 = np.percentile(sim_irs[~np.isnan(sim_irs)], [2.5, 50, 97.5])
    print("\n=== IR UNCERTAINTY (bootstrap CI) ===")
    print(f"IR 95% CI (2.5/50/97.5): {ir_ci_95[0]:.3f}, {ir_ci_95[1]:.3f}, {ir_ci_95[2]:.3f}")

# ============================================================
# SIGNIFICANCE TESTS USING A NULL (mean=0) + block bootstrap
# ============================================================
rets_centered = rets - np.mean(rets)

sim_sharpes_null0 = np.zeros(N_SIMS, float)
for i in range(N_SIMS):
    sim0 = block_bootstrap(rets_centered, n_rets, BLOCK_SIZE, rng)
    sim_sharpes_null0[i] = ann_sharpe(sim0)

p_sharpe_one = float(np.mean(sim_sharpes_null0 >= true_sharpe))
p_sharpe_two = two_sided_p_from_null(sim_sharpes_null0, true_sharpe)

print("\n=== SHARPE SIGNIFICANCE (null: mean=0) ===")
print(f"Observed Sharpe: {true_sharpe:.4f}")
print(f"p-value one-sided (Sharpe > 0): {p_sharpe_one:.4f}")
print(f"p-value two-sided (Sharpe != 0): {p_sharpe_two:.4f}")

if have_spy:
    ex_centered = ex_rets - np.mean(ex_rets)
    sim_ir_null0 = np.zeros(N_SIMS, float)
    for i in range(N_SIMS):
        sim0 = block_bootstrap(ex_centered, len(ex_centered), BLOCK_SIZE, rng)
        sim_ir_null0[i] = ann_sharpe(sim0)

    p_ir_one = float(np.mean(sim_ir_null0 >= true_ir))
    p_ir_two = two_sided_p_from_null(sim_ir_null0, true_ir)

    print("\n=== IR SIGNIFICANCE (excess vs SPY, null: mean=0) ===")
    print(f"Observed IR: {true_ir:.4f}")
    print(f"p-value one-sided (IR > 0): {p_ir_one:.4f}")
    print(f"p-value two-sided (IR != 0): {p_ir_two:.4f}")

# ============================================================
# SUMMARY STATS
# ============================================================
stats = {
    "CAGR_mean": np.nanmean(sim_cagrs),
    "CAGR_median": np.nanmedian(sim_cagrs),
    "CAGR_5pct": np.nanpercentile(sim_cagrs, 5),
    "CAGR_95pct": np.nanpercentile(sim_cagrs, 95),

    "Sharpe_mean": np.nanmean(sim_sharpes),
    "Sharpe_5pct": np.nanpercentile(sim_sharpes, 5),
    "Sharpe_95pct": np.nanpercentile(sim_sharpes, 95),

    "DD_mean": np.nanmean(sim_dds),
    "DD_5pct": np.nanpercentile(sim_dds, 5),
    "DD_95pct": np.nanpercentile(sim_dds, 95),

    f"Prob_MaxDD_ge_{int(DD_THRESHOLD*100)}pct": float(np.mean(sim_dd50)),
    "Prob_CAGR_lt_0": float(np.mean(sim_cagrs < 0)),
    "Prob_Sharpe_lt_0": float(np.mean(sim_sharpes < 0)),
    "Prob_Sharpe_lt_1": float(np.mean(sim_sharpes < 1)),
}

print("\n=== MONTE CARLO STATISTICS (WFO) ===")
for k, v in stats.items():
    print(f"{k}: {v:.4f}")

print("\nPercentile of True CAGR:", float(np.mean(sim_cagrs <= true_cagr)))
print("Percentile of True Sharpe:", float(np.mean(sim_sharpes <= true_sharpe)))

# ============================================================
# DOLLAR DRAWDOWN ANALYSIS (now real $)
# ============================================================
print("\n=== DOLLAR DRAWDOWN ANALYSIS (scaled to START_CAPITAL) ===")
print(f"Start capital: ${START_CAPITAL:,.0f}")
print(f"Median max $ drawdown: ${np.median(sim_max_dd_dollars):,.0f}")
print(f"95th pct max $ drawdown: ${np.percentile(sim_max_dd_dollars, 95):,.0f}")
print(f"Worst case max $ drawdown: ${sim_max_dd_dollars.max():,.0f}")

# Drawdown in $ as % of peak (more meaningful than % of start when equity grows)
dd_pct_of_peak = sim_max_dd_dollars / np.maximum(sim_peak_equity, 1e-12)
print(f"\nMedian max DD as % of peak: {np.median(dd_pct_of_peak)*100:.1f}%")
print(f"95th pct max DD as % of peak: {np.percentile(dd_pct_of_peak,95)*100:.1f}%")

late_bad = np.mean((sim_peak_equity >= LATE_GROWTH_MULT * START_CAPITAL) &
                   (sim_max_dd_dollars >= LATE_DD_FRAC * sim_peak_equity))
print(f"\nProb(≥{int(LATE_DD_FRAC*100)}% DD after +{int((LATE_GROWTH_MULT-1)*100)}% growth): {late_bad:.3f}")

# “Ruin” style metrics (pick your definition)
ruin_50 = float(np.mean(sim_min_equity <= 0.50 * START_CAPITAL))
ruin_25 = float(np.mean(sim_min_equity <= 0.25 * START_CAPITAL))
print(f"\nProb(min equity <= 50% of start): {ruin_50:.4f}")
print(f"Prob(min equity <= 25% of start): {ruin_25:.4f}")

# ============================================================
# SAVE CSV + PLOTS
# ============================================================
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

results = pd.DataFrame({
    "sim_cagr": sim_cagrs,
    "sim_sharpe": sim_sharpes,
    "sim_maxdd": sim_dds,
    "sim_maxdd_dollars": sim_max_dd_dollars,
    "sim_peak_equity": sim_peak_equity,
    "sim_trough_equity": sim_trough_equity,
    "sim_final_equity": sim_final_equity,
    "sim_min_equity": sim_min_equity,
    "sim_sharpe_null_mean0": sim_sharpes_null0,
})

if have_spy:
    results["sim_ir_excess_vs_spy"] = sim_irs

out_csv = os.path.join(OUT_DIR, f"16-mc_wfo_results_{timestamp}.csv")
results.to_csv(out_csv, index=False)
print(f"\nSaved Monte Carlo results → {out_csv}")

def save_hist(data, title, filename, true_value=None):
    plt.figure(figsize=(10, 5))
    data = np.asarray(data, float)
    plt.hist(data[~np.isnan(data)], bins=60, alpha=0.7)
    if true_value is not None:
        plt.axvline(true_value, linewidth=2, label="Actual")
        plt.legend()
    plt.title(title)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, filename))
    plt.close()

save_hist(sim_cagrs, "WFO CAGR Distribution (block bootstrap)", f"mc_wfo_cagr_{timestamp}.png", true_cagr)
save_hist(sim_sharpes, "WFO Sharpe Distribution (block bootstrap)", f"mc_wfo_sharpe_{timestamp}.png", true_sharpe)
save_hist(sim_dds, "WFO Max Drawdown Distribution (block bootstrap)", f"mc_wfo_dd_{timestamp}.png", true_dd)
save_hist(sim_sharpes_null0, "WFO Sharpe Null (mean=0) Distribution", f"mc_wfo_sharpe_null_mean0_{timestamp}.png", true_sharpe)

if have_spy:
    save_hist(sim_irs, "WFO IR (excess vs SPY) Distribution (block bootstrap)", f"mc_wfo_ir_{timestamp}.png", true_ir)

print("\nAll plots saved.")
print("\n=== COMPLETE ===")
# === Custom probability question: equity < $2.5M after 12 years ===
TARGET_YEARS = 25
TARGET_VALUE = 3000_000

# Convert years to fraction of sample length
years_total = (pd.Timestamp(end_date) - pd.Timestamp(start_date)).days / 365.25
frac = TARGET_YEARS / years_total
steps = int(frac * n_rets)

below_count = 0
for i in range(N_SIMS):
    sim_rets = block_bootstrap(rets, n_rets, BLOCK_SIZE, rng)
    sim_curve = np.cumprod(np.r_[1.0, 1.0 + sim_rets]) * START_CAPITAL
    val_at_12y = sim_curve[min(steps, len(sim_curve) - 1)]
    if val_at_12y < TARGET_VALUE:
        below_count += 1

p_below = below_count / N_SIMS
print(f"\nProb(final equity < ${TARGET_VALUE:,.0f} after {TARGET_YEARS} years): {p_below:.4f}")



=== Loading WFO equity curve ===
Loaded 5,033 equity points, 5032 return observations.
Date range: 2005-01-03 → 2024-12-31
Scaled start equity: $360,000

=== TRUE WFO RESULTS (single realized path) ===
CAGR:   0.1729
Sharpe: 0.9820
MaxDD:  -0.2991
IR (Sharpe of excess vs SPY): 0.3580

=== Running Monte Carlo simulations (block bootstrap) ===

=== SHARPE UNCERTAINTY (bootstrap CI) ===
Sharpe 95% CI (2.5/50/97.5): 0.570, 1.012, 1.458

=== IR UNCERTAINTY (bootstrap CI) ===
IR 95% CI (2.5/50/97.5): -0.021, 0.369, 0.754

=== SHARPE SIGNIFICANCE (null: mean=0) ===
Observed Sharpe: 0.9820
p-value one-sided (Sharpe > 0): 0.0000
p-value two-sided (Sharpe != 0): 0.0000

=== IR SIGNIFICANCE (excess vs SPY, null: mean=0) ===
Observed IR: 0.3580
p-value one-sided (IR > 0): 0.0369
p-value two-sided (IR != 0): 0.0727

=== MONTE CARLO STATISTICS (WFO) ===
CAGR_mean: 0.1790
CAGR_median: 0.1784
CAGR_5pct: 0.1065
CAGR_95pct: 0.2532
Sharpe_mean: 1.0127
Sharpe_5pct: 0.6471
Sharpe_95pct: 1.3823
DD_mean: -0.